In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import os
import subprocess
import cyvcf2
import allel

# 设置中文字体支持
from matplotlib import font_manager
font_path = "/home/qmtang/.fonts/NotoSansCJKsc-Regular.otf"          # 刚下载的文件
font_manager.fontManager.addfont(font_path)      # 动态注册
plt.rcParams['font.family'] = font_manager.FontProperties(fname=font_path).get_name()
plt.rcParams['axes.unicode_minus'] = False

1. 筛选东亚+东南亚地区样本

In [2]:
# 读取AADR样本信息
sample_info = pd.read_excel("/mnt/qmtang/EvoFill_data/AADR_1240k/v62.0_1240k_public.xlsx")

# 找到包含'Genetic ID'的列
genetic_id_col = None
for col in sample_info.columns:
    if 'Genetic ID' in col:
        genetic_id_col = col
        print(f"\n找到Genetic ID列: {col}")
        break

if genetic_id_col is None:
    print("未找到Genetic ID列，请检查列名")
else:
    print(f"将使用列名: '{genetic_id_col}'")

# 定义东亚+东南亚地区（修改）
east_southeast_asia_regions = [
    'China', 'Japan', 'Korea', 'Mongolia', 'Taiwan', 'Hong Kong',
    'Thailand', 'Vietnam', 'Myanmar', 'Laos', 'Cambodia', 'Malaysia',
    'Indonesia', 'Philippines', 'Singapore', 'Brunei', 'East Timor'
]

# 筛选东亚+东南亚样本（修改）
east_southeast_asia_samples = sample_info[
    sample_info['Political Entity'].isin(east_southeast_asia_regions)
][genetic_id_col].tolist()

print(f"东亚+东南亚地区样本数量: {len(east_southeast_asia_samples)}")
print("包含的国家/地区:", sorted(sample_info[
    sample_info['Political Entity'].isin(east_southeast_asia_regions)
]['Political Entity'].unique()))

print("前10个样本ID:", east_southeast_asia_samples[:10])


找到Genetic ID列: Genetic ID (suffixes: ".DG" is a high coverage shotgun genome with diploid genotype calls, ".AG" is shotgun data with each position in the genome represented by a randomly chosen sequence, ".HO" is Affymetrix Human Origins genotype data)
将使用列名: 'Genetic ID (suffixes: ".DG" is a high coverage shotgun genome with diploid genotype calls, ".AG" is shotgun data with each position in the genome represented by a randomly chosen sequence, ".HO" is Affymetrix Human Origins genotype data)'
东亚+东南亚地区样本数量: 1917
包含的国家/地区: ['Brunei', 'Cambodia', 'China', 'Indonesia', 'Japan', 'Laos', 'Malaysia', 'Mongolia', 'Myanmar', 'Philippines', 'Taiwan', 'Thailand', 'Vietnam']
前10个样本ID: ['I3611.AG', 'I3612.AG', 'I6221.AG', 'I6222.AG', 'I6224.AG', 'I6225.AG', 'I6226.AG', 'I6228.AG', 'I6231.AG', 'I6232.AG']


2. 提取东亚+东南亚样本的VCF文件

In [3]:
"""提取东亚+东南亚样本"""
# 创建样本列表文件
with open('/mnt/qmtang/EvoFill_data/20251230_chr22/augment/east_southeast_asia_samples.txt', 'w') as f:
    for sample in east_southeast_asia_samples:
        f.write(sample + '\n')

# 提取样本
cmd = f"""
bcftools view -S /mnt/qmtang/EvoFill_data/20251230_chr22/augment/east_southeast_asia_samples.txt \
/mnt/qmtang/EvoFill_data/AADR_1240k/hg38/v62.0_1240k_hg38.chr22.SNPs_only.vcf.gz \
-O z -o /mnt/qmtang/EvoFill_data/20251230_chr22/augment/AADR_east_southeast_asia.chr22.vcf.gz
"""
subprocess.run(cmd, shell=True, check=True)

# 创建索引
subprocess.run("bcftools index /mnt/qmtang/EvoFill_data/20251230_chr22/augment/AADR_east_southeast_asia.chr22.vcf.gz", shell=True, check=True)

print("\n=== 地理分布分析 ===")

# 获取东亚+东南亚样本的地理信息
geo_samples = sample_info[
    sample_info['Political Entity'].isin(east_southeast_asia_regions)
]

# 按国家/地区统计
country_counts = geo_samples['Political Entity'].value_counts()
print("各国家/地区样本数量:")
print(country_counts)



=== 地理分布分析 ===
各国家/地区样本数量:
Political Entity
China          1015
Mongolia        302
Japan           170
Indonesia       149
Vietnam         123
Thailand         77
Taiwan           53
Cambodia         12
Malaysia          5
Laos              4
Philippines       3
Myanmar           2
Brunei            2
Name: count, dtype: int64


In [4]:
os.chdir('/mnt/qmtang/EvoFill_data/20251230_chr22/augment/')

3. 统计VCF基本信息

In [5]:
def get_vcf_stats(vcf_path, label):
    """获取VCF文件的样本数、位点数和缺失率统计"""
    print(f"\n=== {label} 统计信息 ===")
    
    # 使用cyvcf2读取VCF
    vcf = cyvcf2.VCF(vcf_path)
    
    # 获取样本数量
    n_samples = len(vcf.samples)
    print(f"样本数量: {n_samples}")
    
    # 获取样本列表
    sample_list = vcf.samples
    print(f"前5个样本: {sample_list[:5]}")
    
    # 统计位点和缺失率
    variant_count = 0
    missing_counts = []
    
    for variant in vcf:
        variant_count += 1
        # 计算该位点的缺失率
        gt_types = variant.gt_types
        missing_count = np.sum(gt_types == 3)  # -1 表示缺失
        missing_rate = missing_count / n_samples
        missing_counts.append(missing_rate)
        
        if variant_count % 10000 == 0:
            print(f"已处理 {variant_count} 个位点...")
    
    vcf.close()
    
    # 计算统计值
    avg_missing_rate = np.mean(missing_counts)
    median_missing_rate = np.median(missing_counts)
    
    print(f"位点总数: {variant_count}")
    print(f"平均缺失率: {avg_missing_rate:.4f} ({avg_missing_rate*100:.2f}%)")
    print(f"中位缺失率: {median_missing_rate:.4f} ({median_missing_rate*100:.2f}%)")
    
    return {
        'label': label,
        'n_samples': n_samples,
        'n_variants': variant_count,
        'avg_missing_rate': avg_missing_rate,
        'median_missing_rate': median_missing_rate,
        'missing_rates': missing_counts
    }

# 分别统计各个数据集
stats_AADR = get_vcf_stats("AADR_east_southeast_asia.chr22.vcf.gz", "AADR东亚样本")
stats_shimao = get_vcf_stats("shimao_hg38.chr22.vcf.gz", "石峁样本")
stats_tibet = get_vcf_stats("Tibetan300_hg38.chr22.vcf.gz", "西藏样本")


=== AADR东亚样本 统计信息 ===
样本数量: 1917
前5个样本: ['I3611.AG', 'I3612.AG', 'I6221.AG', 'I6222.AG', 'I6224.AG']
已处理 10000 个位点...
位点总数: 16140
平均缺失率: 0.1558 (15.58%)
中位缺失率: 0.0751 (7.51%)

=== 石峁样本 统计信息 ===
样本数量: 147
前5个样本: ['HRR1941463', 'HRR1941464', 'HRR1941465', 'HRR1941467', 'HRR1941468']
已处理 10000 个位点...
位点总数: 15751
平均缺失率: 0.1538 (15.38%)
中位缺失率: 0.0680 (6.80%)

=== 西藏样本 统计信息 ===
样本数量: 300
前5个样本: ['HRR583213', 'HRR583214', 'HRR583215', 'HRR583216', 'HRR583217']
已处理 10000 个位点...
已处理 20000 个位点...
已处理 30000 个位点...
已处理 40000 个位点...
已处理 50000 个位点...
已处理 60000 个位点...
已处理 70000 个位点...
已处理 80000 个位点...
已处理 90000 个位点...
已处理 100000 个位点...
已处理 110000 个位点...
已处理 120000 个位点...
已处理 130000 个位点...
已处理 140000 个位点...
已处理 150000 个位点...
已处理 160000 个位点...
已处理 170000 个位点...
已处理 180000 个位点...
已处理 190000 个位点...
已处理 200000 个位点...
已处理 210000 个位点...
已处理 220000 个位点...
已处理 230000 个位点...
已处理 240000 个位点...
已处理 250000 个位点...
已处理 260000 个位点...
已处理 270000 个位点...
已处理 280000 个位点...
已处理 290000 个位点...
已处理 300000 个位点...
已处理 310000

In [6]:
"""分析位点重叠情况"""
print("\n=== 位点重叠分析 ===")

# 获取各数据集的位点信息
def get_variant_positions(vcf_path):
    positions = set()
    vcf = cyvcf2.VCF(vcf_path)
    for variant in vcf:
        if not len(variant.ALT):
            continue
        pos_key = f"{variant.CHROM}:{variant.POS}:{variant.REF}:{variant.ALT[0]}"
        positions.add(pos_key)
    vcf.close()
    return positions

# 获取位点集合
aadr_variants = get_variant_positions("AADR_east_southeast_asia.chr22.vcf.gz")
shimao_variants = get_variant_positions("shimao_hg38.chr22.vcf.gz")
tibet_variants = get_variant_positions("Tibetan300_hg38.chr22.vcf.gz")

# 计算重叠
overlap_aadr_shimao = aadr_variants & shimao_variants
overlap_aadr_tibet = aadr_variants & tibet_variants
overlap_shimao_tibet = shimao_variants & tibet_variants
overlap_all = aadr_variants & shimao_variants & tibet_variants

print(f"AADR位点数量: {len(aadr_variants)}")
print(f"石峁位点数量: {len(shimao_variants)}")
print(f"西藏位点数量: {len(tibet_variants)}")
print(f"\nAADR与石峁位点重叠: {len(overlap_aadr_shimao)}")
print(f"AADR与西藏位点重叠: {len(overlap_aadr_tibet)}")
print(f"石峁与西藏位点重叠: {len(overlap_shimao_tibet)}")
print(f"三个数据集共同位点: {len(overlap_all)}")

# 计算重叠比例
print(f"\n重叠比例:")
print(f"AADR-石峁: {len(overlap_aadr_shimao)/len(aadr_variants)*100:.1f}%")
print(f"AADR-西藏: {len(overlap_aadr_tibet)/len(aadr_variants)*100:.1f}%")
print(f"石峁-西藏: {len(overlap_shimao_tibet)/len(shimao_variants)*100:.1f}%")


=== 位点重叠分析 ===
AADR位点数量: 16140
石峁位点数量: 15526
西藏位点数量: 400259

AADR与石峁位点重叠: 15517
AADR与西藏位点重叠: 15368
石峁与西藏位点重叠: 15328
三个数据集共同位点: 15326

重叠比例:
AADR-石峁: 96.1%
AADR-西藏: 95.2%
石峁-西藏: 98.7%


4. 合并VCF文件

In [ ]:
def extract_common_variants(overlap_all_positions, input_vcf, output_vcf, label):
    """从VCF文件中提取指定的共有位点"""
    print(f"\n=== 提取 {label} 中的共有位点 ===")
    
    # 创建位点列表文件
    positions_file = f"{label}_common_positions.txt"
    
    with open(positions_file, 'w') as f:
        for pos_key in overlap_all_positions:
            chrom, pos, ref, alt = pos_key.split(':')
            f.write(f"{chrom}\t{pos}\n")
    
    print(f"共有位点数量: {len(overlap_all_positions)}")
    
    # 使用bcftools提取这些位点
    extract_cmd = f"""
    bcftools view -R {positions_file} \
    -O z -o {output_vcf} \
    {input_vcf}
    """
    
    subprocess.run(extract_cmd, shell=True, capture_output=True, text=True)

    # 创建索引
    subprocess.run(f"bcftools index {output_vcf}", shell=True, check=True)
    
    # 验证提取结果
    extracted_count = int(subprocess.check_output(f"bcftools view {output_vcf} | wc -l", shell=True).decode().strip())
    print(f"提取后的位点数量: {extracted_count}")
    
    # 清理临时文件
    os.remove(positions_file)
    
    return output_vcf

# 提取三个数据集中的共有位点
print("=== 提取三个数据集中的共有位点 ===")

common_aadr = extract_common_variants(
    overlap_all,
    "AADR_east_southeast_asia.chr22.vcf.gz",
    "AADR_common.chr22.vcf.gz",
    "AADR"
)

common_shimao = extract_common_variants(
    overlap_all,
    "shimao_hg38.chr22.vcf.gz",
    "shimao_common.chr22.vcf.gz",
    "石峁"
)

common_tibet = extract_common_variants(
    overlap_all,
    "Tibetan300_hg38.chr22.vcf.gz",
    "Tibetan300_common.chr22.vcf.gz",
    "西藏"
)

5. 样本地理分布分析


=== 地理分布分析 ===
各国家/地区样本数量:
Political Entity
China          1015
Mongolia        302
Japan           170
Indonesia       149
Vietnam         123
Thailand         77
Taiwan           53
Cambodia         12
Malaysia          5
Laos              4
Philippines       3
Myanmar           2
Brunei            2
Name: count, dtype: int64


6. 最终汇总表格

In [ ]:
def create_summary_table():
    """创建汇总表格"""
    summary_data = {
        '数据集': ['AADR东亚+东南亚', '石峁', '西藏', '合并后'],
        '样本数量': [stats_AADR['n_samples'], stats_shimao['n_samples'], 
                 stats_tibet['n_samples'], stats_merged['n_samples']],
        '位点数量': [stats_AADR['n_variants'], stats_shimao['n_variants'], 
                 stats_tibet['n_variants'], stats_merged['n_variants']],
        '平均缺失率(%)': [stats_AADR['avg_missing_rate']*100, stats_shimao['avg_missing_rate']*100,
                     stats_tibet['avg_missing_rate']*100, stats_merged['avg_missing_rate']*100],
    }
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

# 显示汇总表
summary_table = create_summary_table()
print("\n=== 数据汇总表 ===")
print(summary_table.to_string(index=False))

# 保存汇总表
summary_table.to_csv('vcf_analysis_summary_east_southeast_asia.csv', index=False, encoding='utf-8-sig')

In [ ]:

"""绘制缺失率分布图"""
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('各数据集缺失率分布', fontsize=16)

datasets = [stats_AADR, stats_shimao, stats_tibet, stats_merged]
positions = [(0,0), (0,1), (1,0), (1,1)]

for i, (stats, pos) in enumerate(zip(datasets, positions)):
    ax = axes[pos]
    
    # 绘制直方图
    ax.hist(stats['missing_rates'], bins=50, alpha=0.7, edgecolor='black')
    ax.set_title(f"{stats['label']}\n平均缺失率: {stats['avg_missing_rate']*100:.2f}%")
    ax.set_xlabel('缺失率')
    ax.set_ylabel('位点数量')
    ax.grid(True, alpha=0.3)
    
    # 添加垂直线显示平均值
    ax.axvline(stats['avg_missing_rate'], color='red', linestyle='--', 
                label=f'平均值: {stats["avg_missing_rate"]*100:.2f}%')
    ax.legend()

plt.tight_layout()
# plt.savefig('missing_rate_distribution.png', dpi=300, bbox_inches='tight')
plt.show()


最终报告

输出文件 : merged_15326_variants_2364_samples.chr22.vcf.gz

- 总样本数 : 2,364个 ✅
- 总位点数 : 15,326个 ✅
- 文件大小 : 28MB
- 平均缺失率 : 16.49%
- 中位缺失率 : 8.12%

### 🔍 样本分布详情

AADR东亚样本 : 1,917个 

#### 按类型
- DG类型: 895个
- AG类型: 744个
- SG类型: 231个
- 其他类型: 47个

#### 按地区
- China          1015
- Mongolia        302
- Japan           170
- Indonesia       149
- Vietnam         123
- Thailand         77
- Taiwan           53
- Cambodia         12
- Malaysia          5
- Laos              4
- Philippines       3
- Myanmar           2
- Brunei            2

石峁样本 : 147个 ✅

西藏样本 : 300个 ✅

总计 : 2,364个样本 ✅